## Meeting Automation LLM agent
We will create an agent which will do following tasks:

- Get information about a prospect & their idea from internet when a call is booked
- Find possible solution about idea from internet
- Create professional project proposal from given idea,client information and solution with other information like tech stack, timeline etc
- Save the project proposal as notion document or word document

First of all, let’s setup our LLM. we will use gpt-4 model for this example

In [ ]:
import constants # create your own constants.py file to store api keys
from langchain.chat_models import ChatOpenAI

# OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or 'OPENAI_API_KEY'
OPENAI_API_KEY = constants.OPENAI_API_KEY

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0.6,
    model_name='gpt-4'
)

gpt-4 is a conversational model, so it will store all previous conversations in a local memory to get the context for current message so it takes the previous messages and sends the full conversation as a context in prompt so that model can understand the context for current message but suppose if there are more than 100 messages then the context will become so much longer and it won’t fit in context window of model. That’s why we will only send last k number of conversations in context and to do that we can use ConversationBufferWindowMemory method with k parameter.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

Now let’s create our first tool which is platform info extractor. To create any custom tool in langchain, you will have to create a class with following properties:

- **name**: name of the tool
- **description**: description about the tool so that agent will know when to use this tool
- **_run()**: this method will be triggered when agent uses the tool so add your business logic in this method
- **_arun()**: this method will be triggered when someone calls agent asynchronously.
We will have to inherit properties of BaseTool class which comes from langchain.tools

We will use perplexity api to get the information from internet and for that we will use their online model `pplx-70b-online`. You can get your own api key from perplexity dashboard.

here is the code for platform info extractor tool:

In [ ]:
# Perplexity setup
from langchain.tools import BaseTool
# Creating tool to get information about given website or platform
import requests
import json
class GetClientDetails(BaseTool):
    name = "Client details extractor"
    description = "use this tool when you have given a platform name, idea or platform link and you want to find more information about CEO of platform and proposed idea"
    def _run(self, website):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "pplx-70b-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You are artificial intelligence agent which extracts an information by searching about it online"
                        "and returns information in this format if it exists"
                        "Title of website or platform: "
                        "Proposed idea:"
                        "CEO Information:"
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "I am giving you platform name,idea or platform link and you have to find latest information about" 
                        "the platform by going to given link and return atleast 200 words description about idea and"
                        "atleast 150 words description about CEO, their goal and achievements"
                        "Please try to make it as detailed as you can and always refer to online information"
                        "here is the platform link or name "
                        f"{str(website)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers)
        return response
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

Now let’s create our second tool which is client details extractor tool:

In [ ]:
from langchain.llms import OpenAI as OpenAI_Agent
class GetPlatformInfo(BaseTool):
    name = "Platform info extractor"
    description = "use this tool when you have given a description about meeting and you have to find the proposed idea, client name or platform link from given description"

    def _run(self, description):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "pplx-70b-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have given a description of meeting and you will have to find out proposed idea, client name and website link mentioned in the given description."
                        "Use client info extractor tool to get more information."
                        "Return it in this format"
                        "Idea: <idea>"
                        "Client Info: <client_info>"
                        "Platform Link: <platform_link>"
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "if anything is not provided in description then use internet and google search to find out the actual information"
                        "Here is the meeting description " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers)
        return response
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

Now let’s create our last tool called solution extractor which will take the idea and give us possible solution for the given idea or problem statement

In [ ]:

class GetIdeaSolution(BaseTool):
    name = "Solution extractor"
    description = "use this tool when you have given an idea description and information about client and you have to find solution on how we can achieve the given idea"

    def _run(self, description):
        url = "https://api.perplexity.ai/chat/completions"
        payload = {
            "model": "pplx-70b-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        "You have given an idea description and you have to find around 3-4 solutions to achieve the given idea using AI-ML"
                        "or webdev technologies. Search online about tech stack, resources, timeline of project and youtube videos and send it with proper formatting and line breaks in this format"
                        "Solution: <solution>"
                        "Tech stack: <tech_stack>"
                        "Timeline: <timeline>"
                    ),
                },
                {
                    "role": "user",
                    "content": (
                        "Here is the idea description " 
                        f"{str(description)}"
                    )
                },
            ]
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "Authorization":f"Bearer {constants.PERPLEXITY_API_KEY}"
        }
        response = requests.post(url, json=payload, headers=headers)
        return response
    
    def _arun(self, website):
        raise NotImplementedError("This tool does not support async")

Now let’s initialize our agent and add all the tools in it

In [ ]:
# Creating Agent
from langchain.agents import initialize_agent
tools = [GetClientDetails(),GetPlatformInfo(),GetIdeaSolution()]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory,
    handle_parsing_errors=True
)

Now it’s time to test our agent 👀 !!

In [ ]:
para = agent(
      "First find the idea, client name and platform link using Platform info extractor tool and find 150 words information about proposed idea about product, goals and achievements of CEO. Once you got the information about their idea then find how we can help them to build it in 200 words"
      "fetch realtime data from internet everytime"
      "This information is going to be added in project proposal so please write in detail and sections like CEO_Info,idea and solution must be explained in 600 words each."
      "Return your response in python dict format like below and please use proper line breaks: "
      """
      "platform_name": platform_name
      "CEO_Name": ceo_name
      "idea": idea
      "solution": solution
      "tech_stack": tech stack
      "timeline": timeline
      "Platform_link": platform_link
      """
      "Here is the description:"
      "Hey there, myself rohan and we are hosting this meeting to discuss about my idea of making a twitter automation tool with several features like automated tweets, scheduled tweets, tweet improvement guides etc"
      )
output = json.loads(para["output"])
print(output)

Now let’s use gpt-4 to format this details so that it can be added in notion as a project proposal (This step is optional).

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=constants.OPENAI_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"""
                I am giving you some data in python dict format and you will have to add more information by searching about it online in given data so that it can be added in project proposal but don't add any fake information. Once you got the information then return the data in the same format.
                Try to add more information by yourself too to make it more detailed and make it around 1000 words. Don't return anything except this dictionary and also please use proper line breaks in every paragraph of each section of dict.
                Here is the information about fields in the given dict: 
                platform_name: Name of platform
                CEO_Info: Background about CEO and information about CEO
                CEO_Name: Name of CEO
                idea: project idea
                solution: solution on how we can solve the given problem statement and how to achieve the given idea
                tech_stack: tech stack used to build the solution
                timeline: timeline for the project
                Platform_link: link to the platform
                Here is the data: 
                {para["output"]}
            """,
        }
    ],
    model="gpt-4",
    temperature=0.7
)
output = chat_completion.choices[0].message.content
output = json.loads(output)

### Adding data to notion
Now we have all the information about client, their idea and organization, so let’s format it to make it as a project proposal and add it in notion as a notion document. we will use `notion_client` module from notion.

Start by installing the module

In [ ]:
!pip install notion_client

Initialize the client

In [ ]:
# Adding data into notion
from notion_client import Client

notion = Client(auth=constants.NOTION_API_KEY)

Now let’s write code to add all our data into notion document:

In [ ]:
from pprint import pprint
parent = {"type": "page_id","page_id": "14ffc7f5-774f-46d6-8653-e6f24dfae758"}
properties = {
    "title": {
        "type": "title",
        "title": [{ "type": "text", "text": { "content": f"Meeting with {output['CEO_Name']} about {output['platform_name']}" } }]
    },
}
children = [
      {
        "object": "block",
        "type": "heading_2",
        "heading_2": {
          "rich_text": [{ "type": "text", "text": { "content": "🚀 Platform Name" } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['platform_name'] } }]
        }
      },
      {
        "object": "block",
        "type": "heading_2",
        "heading_2": {
          "rich_text": [{ "type": "text", "text": { "content": "📝 Idea Description" } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['idea'] } }]
        }
      },
      {
        "object": "block",
        "type": "heading_2",
        "heading_2": {
          "rich_text": [{ "type": "text", "text": { "content": "💡 How we can help?" } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['solution'] } }]
        }
      },
      {
        "object": "block",
        "type": "heading_3",
        "heading_3": {
          "rich_text": [{ "type": "text", "text": { "content": "⚒️ What tech stack we can use?" } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['tech_stack'] } }]
        }
      },
      {
        "object": "block",
        "type": "heading_3",
        "heading_3": {
          "rich_text": [{ "type": "text", "text": { "content": "⌛ What is timeline of project? " } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['timeline'] } }]
        }
      },
      {
        "object": "block",
        "type": "heading_3",
        "heading_3": {
          "rich_text": [{ "type": "text", "text": { "content": "🤔 Info about CEO" } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['CEO_Info'] } }]
        }
      },
      {
        "object": "block",
        "type": "heading_2",
        "heading_2": {
          "rich_text": [{ "type": "text", "text": { "content": "🔗 Platform Link " } }]
        }
      },
      {
        "object": "block",
        "type": "paragraph",
        "paragraph": {
          "rich_text": [{ "type": "text", "text": { "content": output['Platform_link'] } }]
        }
      },
    ]
create_page_response = notion.pages.create(
    parent=parent,properties=properties,children=children
)
pprint(create_page_response)

### Creating proposal as a word document
The main problem with notion formatting is that everything needs to be converted into notion object which you already saw above in code and you can’t write an object for every single paragraph, list or heading to make your document good so instead of using notion we will try to get an output in markdown from our agent and then convert that text into an actual word document.

Let’s change the part where we made our last openai call for formatting. Instead of getting python dictionary response, we will tell model to give response in markdown

Make these changes in your previous code:

In [ ]:
# Code for creating a markdown for our proposal
from openai import OpenAI
client = OpenAI(api_key=constants.OPENAI_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"""
                I am giving you some data in python dict format and you will have to add more information in it and convert it in markdown file with below format
                Don't use anything except headings and paragraphs in markdown
                # Project name
                project name
                # Who is the client?
                information about client
                # What is the idea?
                information about idea
                # How can we help?
                detailed information about solution
                # Tech stack
                information about tech stack
                # Timeline
                information about timeline
                Try to add more information by yourself too to make it more detailed and make it around 800 words. Don't return anything except markdown and use proper line breaks.
                Here is the information about fields in the given dict: 
                platform_name: Name of platform
                CEO_Info: Background about CEO and information about CEO
                CEO_Name: Name of CEO
                idea: project idea
                solution: solution on how we can solve the given problem statement and how to achieve the given idea
                tech_stack: tech stack used to build the solution
                timeline: timeline for the project
                Platform_link: link to the platform
                Here is the data: 
                {para["output"]}
            """,
        }
    ],
    model="gpt-4",
    temperature=0.7
)
output = chat_completion.choices[0].message.content
print(output)
# output = json.loads(output)
# print(json_output["output"])

And now we have our project proposal as a markdown text so now its time to convert it into word document and we can easily do that with pypandoc which is a python wrapper for pandoc.

Let’s install it

In [ ]:
!pip install pypandoc

Now we just have to provide our markdown text and it will convert it into word document. So let’s try it out!

In [ ]:
import pypandoc
output = pypandoc.convert_text(output, 'docx', format='md', outputfile="output.docx")
# output = json.loads(output.output)
print(output)